<a href="https://colab.research.google.com/github/Johyeyoung/DATA/blob/master/Trigger_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 단어 리스트만들기

## 라이브러리 설치

In [4]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기
!pip install krwordrank # 키워드 추출기

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graph

## 데이터 가져오기

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/2-2._lesion_info_table.csv', encoding='cp949')

In [5]:
# 같은 내용의 카테고리 합치기
df["lesion_detail_type"] = df["lesion_detail_type"].str.replace(pat=r'하이패스_분실', repl=r'하이패스_카드분실', regex=True)

# 3차 level의 종류
inspect_type=df['lesion_detail_type']

inspect_type=set(inspect_type.values.tolist())
print(inspect_type)
print(len(inspect_type))

{'창문_작동불가', '위생_워셔액', '블랙박스_분실', '위생_담배(담배냄새, 담뱃재 등)', '하이패스_카드오류', '와이퍼_교체필요', '도어_소음', '에어컨_소음', '단말기_파손', '와이퍼_고장', '휠_손상', '후방카메라_파손(탈거, 각도이상)', '주차장_중계기점검필요', '차량상태확인', '경고등_연료부족', '후방카메라_고장', '주차장_입차불가(만차)', '충전카드_분실', '사이드미러_고장', '주행_차량떨림', '브레이크_고장', '외관_파손(상부)', '위생_외부위생불량 (유리, 경고장 등)', '하이패스_파손(탈거, 파손)', '주행_변속레버파손', '위생_악취', '경고등_브레이크', '주행_핸들떨림', '블랙박스_고장', '타이어_파스/펑크(전좌)', '라이트/전구_경고등(고장)', '와이퍼_파손', '룸미러_파손', '타이어_마모', '오디오_버튼파손/분실', '충전카드_보관함 파손/탈거', '내비게이션_파손', '창문_파손', '경고등_온도게이지', '사이드미러_파손', '단말기_신호불량', '도어_파손', '위생_내부위생불량(모래, 흙, 먼지 등)', '타이어_코드절상', '핸들_기능조작', '주행_소음', '주행_변속기이상', '번호판_분실', '충전카드_결제오류', '위생_쓰레기', '사고조사_요청', '단말기_배선', '경고등_스패너(서비스알림)', '트렁크_고장', '핸들_소음', '경고등_에어백', '주유카드_결제오류', '시동_배터리방전', '시거잭_고장', 'ADAS_파손', '내비게이션_업데이트', '히터_고장', '문제어불가_옥스트라', '경고등_엔진오일', '주유구_고장/파손', '거치대_파손', '워셔액_(통)파손', '안심번호_연동오류', '히터_소음', '외부스티커_훼손', '글로브박스_파손', '사이드브레이크_해제불가', '주행_차량쏠림(얼라이먼트)', '내비게이션_인터넷연결불가', '번호판_파손', '위생_구토', '시동_키인식불가(키아웃)', '타이어_공기압경고등', '워셔액_고장', '후방

### 각 원인별 데이터

In [6]:
def customer_info():
    phone_number='(인입번호|[연락]*[\s]*가능[\s]*한[\s]*번호)[^\n]*'
    url='(관제\s예약\s)*URL[주소:\s]*[^\n]+'
    location='(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*'
    lst = [phone_number, url, location]
    return '|'.join(lst)

def service_info():
    case_number='([원]*예약번호|블락번호)[^\n]*'
    card='(사고카드|장애카드|장애블락)[\s][^\n]*' # 장애카드 생성, 장애블락 생성
    picture='(사진[\s]*확인|사진[\s]*여부)[^\n]*' # 사진 여부 | 확인 
    customer='((고객[\s]*과실[\s]*(사유|여부))|(현장[\s]*고객[\s]*유무)|([^\n]*본인[\s]*확인)|(고객운행중))[^\n]*'
    chatting='([\\n]*(채팅[\s]*상담|톡)[\s]*[^\n]*)|([\\n]*선제[(조치)(장애)(건)\s]*)|CC'  # 채팅, 톡 /선제조치건/ 사진여부 /본인확인 제거
    lst = [case_number, card, picture, customer, chatting]
    return '|'.join(lst)

    #groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'([\\n]*(채팅[\s]*상담|톡)[\s]*[^\n]*)|([\\n]*선제조치[\s]*(장애)*[\s]*(건)*)', repl=r'', regex=True)
    #groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'([^\n]*본인[\s]*확인[^\n]*)', repl=r'', regex=True)
    #groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'관제\s예약\sURL\s주소:[\s]*http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),|]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', repl=r'', regex=True)
    #groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'ㄴ\s수긍|ㄴ수긍[/s]*후?|[원]*예약번호[\s]*:[\s]*|(인입번호[:|\s]*)|CC|(연락[\s]*가능[\s]*한[\s]*번호:[\s]*)', repl=r'', regex=True)
    #groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*', repl=r'', regex=True)


In [7]:
import re

dic_trigger={}
inspect_num=0
# lesion_detail_type 그룹별 데이터 확인
for inspect in inspect_type:
  inspect_num+=1
  #print(inspect_num)
  #if inspect_num<5: 
  trigger=[]
  figure=[]
  groupdf=df.groupby('lesion_detail_type').get_group(inspect) 
  lesion_id=groupdf['lesion_id']
  lesion_id=set(lesion_id.values.tolist())
  for id in lesion_id:
    groupdf2=groupdf.groupby('lesion_id').get_group(id).sort_values(by='memo_id') # 시간순으로 나열 
    groupdf2=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf3=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'ㅁ', repl='\nㅁ\n', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(\n+)', repl=r'\n', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z:\s]', repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(문의[\s]?구분[\s]*[아래\s중\s택1]*[\\n]*[ㄴ]*)|(문의[\s]?내용[\s]*[ㅁ|ㄴ]*)', repl=r'문의내용: ', regex=True)  # '문의 구분 택'의 경우
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[\s]*:[xXo0O/(완료)(확인불가) ]*', repl=r':', regex=True) # 여부 ox표시
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=customer_info(), repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=service_info(), repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'부위[\s]*:', repl=r'부위는 ', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'긴출', repl=r'긴급출동', regex=True)

    memo=str(groupdf2['memo'].values.tolist()[0])  # 첫부분 원소가- Trigger
    memo2=str(groupdf3['memo'].values.tolist()[0])  # 첫부분 원소가- Trigger


    # figure_txt = str(groupdf2['memo'].values.tolist()[1:])  # 다음 원소가- Figure
    # print(figure_txt)
    memo_list=memo.split('ㅁ')
    origin_list=memo2.split('ㅁ')
    #print(origin_list)
    #print(memo_list)
    factor_trgr = inspect.replace('_', '|') + '|고장|파손|문의|증상(\s+)상세'
    factor_trgr = re.compile(factor_trgr)
    factor_fig = re.compile(r'약속|안내|처리')
    flag = 0
    for i in memo_list:
      m = factor_trgr.search(i)
      p = factor_fig.search(i)
      if p:
        figure.append(i)
        #print('-------', i)
        flag = 1
      else:
        if m and flag == 0:
          trigger.append(i)
          #print('+++++++', i)
        else:
          figure.append(i)
        # print('-------', i)
  dic_trigger[inspect_num]=trigger  

In [8]:
# 형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt
okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls



# 특수문자 제거
import re
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)



# 맞춤법, 띄어쓰기 교정 - 텍스트 전처리
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

In [9]:
# csv 파일로 저장하기 
# import csv
# f = open('/content/drive/MyDrive/Trigger_list.csv','w', newline='\n')
# wr = csv.writer(f)
# wr.writerow(['type', '문의 내용'])



f = open('/content/drive/MyDrive/Trigger_list.txt','w')


# (사건 타입 번호 - 문의 사항) 저장
for idx, trigger in dic_trigger.items():
  trigger = set(trigger) # 1차 중복 제거
  
  # \n으로 나누기 전 문장 내에 \n으로 끊길 가능성 있는거 이어주기
  trigger_str= '\n'.join(trigger)
  trigger_str=re.sub(r'(\n+)', '\n', trigger_str)
  trigger_str=re.sub(r'\nㄴ|ㄴ', ' ', trigger_str)
  trigger_str=re.sub(r'(손상고장[\s][부위는]*|파손고장[\s][부위는]*|문의내용|문의사항|증상[\s]*상세내용|접수[\s]*상세[\s]*사유|접수내용|조치사항|확인사항|확인내용|현장조치)[:\s]*', '', trigger_str)
  trigger_str=re.sub(r'(여부[oOxX:]*)', '', trigger_str)
  trigger_str=re.sub(' +', ' ', trigger_str) # 다중 공백 제거
  trigger_str=re.sub(r'( )*\n( )*', '\n', trigger_str)
  trigger_str=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\\n ]|(긴급출동 접수)\n', '', trigger_str) 
  trigger_str=re.sub('\n', '-', trigger_str) 

  #keyword_tracker2(trigger_str) # 형태소 추출기로 하면 정확도가 낮음 
  #trigger_str=correction(trigger_str)
  #print(trigger_str)
  # 맞춤법 검사기 돌리기 위해 특수문자 제거
  b = list(set(trigger_str.split('-')))

  print(sorted(b))
  b.sort(key=lambda x:len(x))
  #print(idx, b)


  for B in b:
    if len(B) > 3:
      #id\tdocument\tlabel\n : 형식
      # wr.writerow([idx, B]) # cs
      B=correction(B)
      print(B)
      f.write(str(idx)+'\t'+B+'\t'+str(idx)+'\n') # 각 리스트에 담긴 문장을 쓴다.

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
내비게이션이 안된다고 하심
내비게이션이 되지 않음으로 인입
내비게이션 하단이 잘려서 보임
호차량 네비 업데이트 오류 확인 요청
내비게이션 SD카드 업데이트 오류
변경 받을 쏘카 존에 도착하였다며 인입
내비게이션 블루투스 작동 문의 인입
사이드브레이크가 해제되지 않아 인입
내비게이션 오작동 후방카메라 오작동
계속 업데이트와 재부팅을 반복합니다
내비게이션 업데이트 오류로 인해 인입
호 이동 주차 필요 지정 주차장 물청소
내비에 지도 데이터가 없다고 뜬다며 인입
내비 sd카드 삽입하라 해서 차량 변경해달라 하심
내비게이션 sd카드 업데이트 문제로 인입
고객님 내비게이션 업데이트 불량 사용 불가 인입
내비 sd카드가 오류 메시지 확인된다고 문의
내비게이션이 TMAP 지연 중이라며 안된다고 함
아틀란 소리 이용 불가 패드에서 소리가 안 나옴
전원은 연결되나 SD카드 업데이트 메시지 출현
내비게이션과 와이파이가 작동이 안 돼서 문의드림
티맵 준비 중입니다 화면에서 멈춰서 움직이지 않음
내비게이션 sd카드 업로드 화면 보이며 이용불가
내비게이션 T맵 실행 중 멘트에서 동일 화면 유지됨
업데이트 오류라고 하면서 내비 작동이 되지 않음
네비 업데이트하라고 나온 이후 터치 불가 네비 작동이 안 됨
내비가 운행 중 갑자기 업데이트된다며 조치 요청
지우는 중이라는 메시지 노출로 네비 이용 불가 상태
내비게이션 sd카드 삽입 후 재부팅 멘트 뜬다고 함
고객님 내비게이션 업데이트 오류 후방카메라 사용 불가 인입
내비게이션이 업데이트 오류라고 나오고 아무것도 작동하지 않음
내비게이션이 최적화돼야 된다며 자꾸 문구 발생되어 인입
내비게이션 실행 중입니다라고 문구 켜지고 안 넘어가서 인입
내비 업데이트 오류로 점검 요청 운행은 다 하고 반납하심
내비게이션 SD카드 오류 문구 뜬다 업데이트 오류 발생 뜬다
업데이트해야 한다고 업데이트 이후 또 업데이트 시작 무한 반복이라고 함
내비게이션 업데이트 오류로 음 곧 반납으로 차량 이용

In [10]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
def keyword_tracker(trigger):
  texts = [normalize(text, english=True, number=True) for text in trigger]  # 문장내 특수문자 제거하기
  texts
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 10, # 단어의 최대 길이
      verbose = True
      )

  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10

  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
      print('%8s:\t%.4f' % (word, r))
  return

In [11]:
def keyword_tracker2(memo):
  memo=sub_special(memo) # memo-문자열
  memo=morph_and_stopword(memo)
  memo=Counter(memo)
  memo_rank=memo.most_common(100)
  print(memo_rank)
  return

description=str(set(groupdf['description'].values.tolist())) 
description=sub_special(description)
description=morph_and_stopword(description)
description=Counter(description)
description_rank=description.most_common(100)


print(description_rank)
# 형태소 분리
#print('memo:\n', morph_and_stopword(memo))
#print('description:\n', morph_and_stopword(description))


[('시동', 9), ('꺼지다', 7), ('점검', 6), ('가산', 5), ('IT', 5), ('교환', 5), ('주행', 5), ('없다', 5), ('블루', 4), ('지프', 4), ('강남', 4), ('서비스', 4), ('센터', 4), ('보충', 4), ('HK', 4), ('양재', 4), ('시운전', 3), ('정점', 3), ('바디', 3), ('보증', 3), ('대행', 3), ('690680', 3), ('장애', 3), ('차량', 3), ('주차', 3), ('장내', 3), ('테스트', 3), ('이상', 3), ('송파', 2), ('배명', 2), ('현상', 2), ('대납', 2), ('AS', 2), ('교체', 2), ('엔진오일', 2), ('완료', 2), ('배터리', 2), ('단자', 2), ('체결', 2), ('확인', 2), ('처리', 2), ('탁송', 2), ('용산', 2), ('산천', 2), ('운앞', 2), ('밉다', 2), ('문제', 2), ('되다', 2), ('입다', 2), ('요청', 2), ('좌우', 2), ('우측', 1), ('전조등', 1), ('스테이션', 1), ('하남', 1), ('진단', 1), ('스캐너', 1), ('도림', 1), ('정차', 1), ('위반', 1), ('과태료', 1), ('납부', 1), ('진행', 1), ('프레', 1), ('어셈블리', 1), ('에어컨', 1), ('냉매', 1), ('회수', 1), ('충전', 1), ('TS', 1), ('동점', 1), ('타다', 1), ('청소', 1), ('스로틀', 1), ('클리닝', 1), ('스피드', 1), ('메이트', 1), ('창점', 1), ('10분', 1), ('이상시운전결과', 1), ('불량', 1), ('정상', 1), ('복귀', 1), ('딜리', 1), ('버리다', 1), ('으로', 1), ('범퍼', 1), ('부분', 1), (

## 객체인식명


In [ ]:
!git clone https://github.com/eagle705/pytorch-bert-crf-ner.git

In [ ]:
!pip3 install torch torchvision
!pip3 install pytorch_pretrained_bert>=0.4.0
!pip3 install mxnet>=1.5.0
!pip3 install gluonnlp>=0.6.0
!pip3 install sentencepiece>=0.1.6
!pip3 install git+https://github.com/kmkurn/pytorch-crf#egg=pytorch_crf
!pip3 install transformers
!pip3 install tb-nightly
!pip3 install future
import os


In [ ]:
!pip3 install Config
!pip3 install Tokenizer

In [ ]:
!python pytorch-bert-crf-ner/inference.py
